In [4]:
import tensorflow
from tensorflow.keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, Dropout, Input, Reshape
from tensorflow.keras.regularizers import l2
import yaml

def loss_wrapper(weights_loss):
    def custom_loss(y_true, y_pred):
        return tf.nn.softmax_cross_entropy_with_logits()
def get_yamldict(yaml_path):
    with open(yaml_path, "r", encoding="utf-8") as yaml_file:
        yaml_dict = yaml.load(yaml_file, Loader=yaml.Loader)
    return yaml_dict
def getModel(word_seq_len, char_seq_len, ngram_seq_len, chars_dict_len, word_dict_len, ngram_dict_len, reg_lambda,
             emb_dim):
    ########### char-level embedding, conv layer #############
    char_input = Input(shape=(char_seq_len), name="char_input")
    print("shape of char_input: ", char_input.shape)
    emb_char = Embedding(chars_dict_len, emb_dim)(char_input)
    print("char-Embedding: ", emb_char.shape)
    pooled_result = []
    for filter_size in [3, 4, 5, 6]:
        char_conv_output = Conv1D(filters=256, kernel_size=filter_size, strides=1, activation="relu", padding="VALID")(
            emb_char)
        print("char-convOutput : ", char_conv_output.shape)
        max_pooled_output = MaxPooling1D(pool_size=char_seq_len  - filter_size + 1, strides=1, padding="VALID")(
            char_conv_output)
        print("char - maxPool shape: ", max_pooled_output.shape)
        pooled_result.append(max_pooled_output)
    num_filters_total = 256 * 4
    filter_pooled = tensorflow.concat(pooled_result, axis=1)
    print("4 filter output pooled: ", filter_pooled.shape)
    char_flat = tensorflow.reshape(filter_pooled, [-1, num_filters_total])
    char_drop_out = Dropout(0.5)(char_flat)
    ######### word-level embedding, conv_layer ###########
    input_word = Input(shape=(word_seq_len), name="word_input")
    input_ngram = Input(shape=(word_seq_len * ngram_seq_len), name="ngram_input")
    reshaping_layer = Reshape((word_seq_len, ngram_seq_len), input_shape=(word_seq_len * ngram_seq_len,))
    reshepd_ngram = reshaping_layer(input_ngram)
    # input_ngram_padded = Input(shape=(200,20,32,), name="ngram_padded_input")
    emb_word = Embedding(word_dict_len + 1, emb_dim)(input_word)
    print("word-embedding: ", emb_word.shape)
    emb_ngram = Embedding(ngram_dict_len + 1, emb_dim)(reshepd_ngram)
    emb_ngram_sum = tensorflow.reduce_sum(emb_ngram, 2)
    tot_word_emb = tensorflow.add(emb_ngram_sum, emb_word)
    word_pooled_result = []
    for filter_size in [3, 4, 5, 6]:
        layer_name = "word_layer_fsize_" + str(filter_size)
        word_conv_output = Conv1D(filters=256, kernel_size=filter_size, strides=1, activation="relu", padding="VALID",
                                  name=layer_name)(tot_word_emb)
        max_pooled_output_word = MaxPooling1D(pool_size=word_seq_len - filter_size + 1, strides=1, padding="VALID")(
            word_conv_output)
        word_pooled_result.append(max_pooled_output_word)
    word_filter_pooled = tensorflow.concat(word_pooled_result, axis=1)
    word_flat = tensorflow.reshape(word_filter_pooled, [-1, num_filters_total])
    print("word_flat shape: ", word_flat.shape)
    word_drop_out = Dropout(0.5)(word_flat)
    ############# Fully connected Layer ##########
    char_dense_layer = Dense(512, input_shape=(num_filters_total,), kernel_initializer="glorot_uniform",
                             kernel_regularizer=l2(reg_lambda))(char_drop_out)
    word_dense_layer = Dense(512, input_shape=(num_filters_total,), kernel_initializer="glorot_uniform",
                             kernel_regularizer=l2(reg_lambda))(word_drop_out)
    conv_final_output = tensorflow.concat([char_dense_layer, word_dense_layer], 1)
    ############### Dense layer - before_softmax #######################
    d0 = Dense(512, input_shape=(1024,), kernel_initializer="glorot_uniform", kernel_regularizer=l2(reg_lambda))(
        conv_final_output)
    d1 = Dense(256, input_shape=(512,), kernel_initializer="glorot_uniform", kernel_regularizer=l2(reg_lambda))(d0)
    d2 = Dense(128, input_shape=(256,), kernel_initializer="glorot_uniform", kernel_regularizer=l2(reg_lambda))(d1)
    score = Dense(2, input_shape=(128,), kernel_initializer="glorot_uniform", kernel_regularizer=l2(reg_lambda))(d2)
    ############### Score & Predictions & Probability ##############
    predictions = tensorflow.argmax(score, 1, name="predictions")
    prob = tensorflow.nn.softmax(score, name="prob")
    model = tensorflow.keras.Model([char_input, input_word, input_ngram], prob)
    return model

def input_fn(path, maxWordPerUrl, maxCharPerUrl, maxNgramPerUrl, shuffle_buffer_size, batch_size):
    dataset = tensorflow.data.TFRecordDataset(path)
    feature_map = {
        "engineeredChar": tensorflow.io.FixedLenFeature((maxCharPerUrl,), tensorflow.int64),
        "engineeredWord": tensorflow.io.FixedLenFeature((maxWordPerUrl,), tensorflow.int64),
        "engineeredNgram": tensorflow.io.FixedLenFeature((maxWordPerUrl * maxNgramPerUrl,), tensorflow.int64),
        "label": tensorflow.io.FixedLenFeature([], tensorflow.int64)
    }
    def _parse_fn(record, feature_map):
        example = tensorflow.io.parse_single_example(serialized=record, features=feature_map)
        onehot_label = tensorflow.cast(tensorflow.one_hot(tensorflow.where(tensorflow.equal(example["label"], 1), 1, 0), depth=2), dtype=tensorflow.int64)
        return {"char_input": example["engineeredChar"], "word_input": example["engineeredWord"],
                "ngram_input": example["engineeredNgram"]}, onehot_label
    dataset = dataset.map(lambda record: _parse_fn(record, feature_map))
    return dataset.shuffle(shuffle_buffer_size).batch(batch_size)
def read_tfrecord(path, maxWordPerUrl, maxCharPerUrl, maxNgramPerUrl, read_batch_size):
    dataset = tensorflow.data.TFRecordDataset(path)
    feature_map = {
        "engineeredChar": tensorflow.io.FixedLenFeature((maxCharPerUrl,), tensorflow.int64),
        "engineeredWord": tensorflow.io.FixedLenFeature((maxWordPerUrl,), tensorflow.int64),
        "engineeredNgram": tensorflow.io.FixedLenFeature((maxWordPerUrl * maxNgramPerUrl,), tensorflow.int64),
        "label": tensorflow.io.FixedLenFeature([], tensorflow.int64)
    }
    def _parse_fn(record, feature_map):
        example = tensorflow.io.parse_single_example(serialized=record, features=feature_map)
        onehot_label = tensorflow.cast(tensorflow.one_hot(tensorflow.where(tensorflow.equal(example["label"], 1), 1, 0), depth=2), dtype=tensorflow.int64)
        return {"char_input": example["engineeredChar"], "word_input": example["engineeredWord"],
                "ngram_input": example["engineeredNgram"]}, onehot_label
    dataset = dataset.map(lambda record: _parse_fn(record, feature_map))
    return dataset.batch(read_batch_size)
 
def input_small_fn(path, maxWordPerUrl, maxCharPerUrl, maxNgramPerUrl):
    dataset = tensorflow.data.TFRecordDataset(path)
    feature_map = {
        "engineeredChar": tensorflow.io.FixedLenFeature((maxCharPerUrl,), tensorflow.int64),
        "engineeredWord": tensorflow.io.FixedLenFeature((maxWordPerUrl,), tensorflow.int64),
        "engineeredNgram": tensorflow.io.FixedLenFeature((maxWordPerUrl * maxNgramPerUrl,), tensorflow.int64),
        "label": tensorflow.io.FixedLenFeature([], tensorflow.int64)
    }
    def _parse_fn(record, feature_map):
        example = tensorflow.io.parse_single_example(serialized=record, features=feature_map)
        onehot_label = tensorflow.cast(tensorflow.one_hot(tensorflow.where(tensorflow.equal(example["label"], 1), 1, 0), depth=2), dtype=tensorflow.int64)
        return {"char_input": example["engineeredChar"], "word_input": example["engineeredWord"],
                "ngram_input": example["engineeredNgram"]}, onehot_label
    dataset = dataset.map(lambda record: _parse_fn(record, feature_map))
    return dataset.take(10)
 
def get_label_and_urlstring(path):
    dataset = tensorflow.data.TFRecordDataset(path)
    feature_map = {
        "url": tensorflow.io.FixedLenFeature([], tensorflow.string),
        "label":tensorflow.io.FixedLenFeature([], tensorflow.int64),
    }
    def _parse_fn(record, feature_map):
        example = tensorflow.io.parse_single_example(serialized=record, features=feature_map)
        one_hot_label = tensorflow.cast(tensorflow.one_hot(tensorflow.where(tensorflow.equal(example["label"], 1),1,0), depth=2), dtype=tensorflow.int64)
        return {"url": example["url"], "label": one_hot_label}
    
    return dataset.map(lambda record: _parse_fn(record, feature_map))
    
    
def get_label_url_source(path):
    dataset = tensorflow.data.TFRecordDataset(path)
    feature_map = {
        "url": tensorflow.io.FixedLenFeature([], tensorflow.string),
        "label":tensorflow.io.FixedLenFeature([], tensorflow.int64),
        "urlSource":tensorflow.io.FixedLenFeature([], tensorflow.string)
    }
    def _parse_fn(record, feature_map):
        example = tensorflow.io.parse_single_example(serialized=record, features=feature_map)
        one_hot_label = tensorflow.cast(tensorflow.one_hot(tensorflow.where(tensorflow.equal(example["label"], 1),1,0), depth=2), dtype=tensorflow.int64)
        return {"url": example["url"], "label": one_hot_label, "urlSource": example["urlSource"]}
    
    dataset = dataset.map(lambda record: _parse_fn(record, feature_map))
    return dataset

In [5]:
import pandas as pd 

In [84]:
def make_dict(path):
    result={}
    tmp = pd.read_parquet(path)
    for token, index in zip(tmp.iloc[:,0], tmp.iloc[:,1]):
        result[index]=token
    return result    

In [85]:
word_dict = make_dict("/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/engineered-data/totalSet/2020-11-09/small_dictionary/wordTokenIndex")



In [78]:
w = pd.read_parquet("/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/engineered-data/totalSet/2020-11-09/small_dictionary/charTokenIndex")

In [8]:
word_seq_len = 200
char_seq_len = 200
ngram_seq_len = 20
chars_dict_len = 1406 
word_dict_len = 10507
ngram_dict_len = 1406
reg_lambda = 0 
emb_dim = 32

In [7]:
model = getModel(word_seq_len, char_seq_len, ngram_seq_len, chars_dict_len, word_dict_len, ngram_dict_len, reg_lambda, emb_dim)

shape of char_input:  (None, 200)
char-Embedding:  (None, 200, 32)
char-convOutput :  (None, 198, 256)
char - maxPool shape:  (None, 1, 256)
char-convOutput :  (None, 197, 256)
char - maxPool shape:  (None, 1, 256)
char-convOutput :  (None, 196, 256)
char - maxPool shape:  (None, 1, 256)
char-convOutput :  (None, 195, 256)
char - maxPool shape:  (None, 1, 256)
4 filter output pooled:  (None, 4, 256)
word-embedding:  (None, 200, 32)
word_flat shape:  (None, 1024)


In [6]:
train_base_dir = "/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/engineered-data/totalSet/2020-11-09/small_splitted/train.tfrecord/part-r-"
train_data_dir = [train_base_dir+str(idx).zfill(5) for idx in range(5)]

val_base_dir =  "/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/engineered-data/totalSet/2020-11-09/small_splitted/test.tfrecord/part-r-"
val_data_dir = [val_base_dir+str(idx).zfill(5) for idx in range(5)]

oop_base_dir = "/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/engineered-data/totalSet/2020-11-09/small_oop/tfrecords"
oop_data_dir = [oop_base_dir + str(idx).zfill(5) for idx in range(5)]

In [9]:
tr_dataset = input_fn(train_data_dir, word_seq_len, char_seq_len, ngram_seq_len, 10000, 256)
val_dataset = input_fn(val_data_dir, word_seq_len, char_seq_len, ngram_seq_len, 10000, 256)
oop_dataset = input_fn(oop_data_dir, word_seq_len, char_seq_len, ngram_seq_len, 10000, 256)



In [6]:
oop_test_dataset = input_small_fn(oop_data_dir, word_seq_len, char_seq_len, ngram_seq_len)

In [9]:
val_dataset.unbatch().take(3)

<TakeDataset shapes: ({char_input: (200,), word_input: (200,), ngram_input: (4000,)}, (2,)), types: ({char_input: tf.int64, word_input: tf.int64, ngram_input: tf.int64}, tf.int64)>

In [10]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.fit(tr_dataset, epochs=3, validation_data=val_dataset)

In [8]:
model.save_weights("/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/model/2020-11-09/small_70k")

NameError: name 'model' is not defined

In [7]:
model = getModel(word_seq_len, char_seq_len, ngram_seq_len, chars_dict_len, word_dict_len, ngram_dict_len, reg_lambda, emb_dim)
model.load_weights("/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/model/2020-11-09/small_70k")

shape of char_input:  (None, 200)
char-Embedding:  (None, 200, 32)
char-convOutput :  (None, 198, 256)
char - maxPool shape:  (None, 1, 256)
char-convOutput :  (None, 197, 256)
char - maxPool shape:  (None, 1, 256)
char-convOutput :  (None, 196, 256)
char - maxPool shape:  (None, 1, 256)
char-convOutput :  (None, 195, 256)
char - maxPool shape:  (None, 1, 256)
4 filter output pooled:  (None, 4, 256)
word-embedding:  (None, 200, 32)
word_flat shape:  (None, 1024)


In [8]:
layers_wt = model.get_layer("word_layer_fsize_3").weights


In [9]:
layers_wt

[<tf.Variable 'word_layer_fsize_3/kernel:0' shape=(3, 32, 256) dtype=float32, numpy=
 array([[[ 4.37490419e-02,  4.04972695e-02,  6.83686808e-02, ...,
          -2.94406768e-02,  2.14401856e-02, -7.07160309e-02],
         [-5.03828079e-02,  8.21285620e-02,  8.11735634e-03, ...,
          -5.17946742e-02,  6.70022368e-02, -5.20262942e-02],
         [-5.69256023e-02,  7.66882300e-02,  2.32748836e-02, ...,
           6.02978878e-02,  6.99732155e-02, -1.93480458e-02],
         ...,
         [ 2.08949335e-02,  8.45661089e-02, -6.97435662e-02, ...,
           3.94282378e-02, -5.96826710e-02,  6.03995733e-02],
         [-7.13708550e-02,  3.20183225e-02, -4.33528656e-03, ...,
           5.30924164e-02, -7.34575838e-02,  4.28760387e-02],
         [ 1.72489919e-02,  6.88444823e-04,  4.73778276e-03, ...,
          -4.49759476e-02, -3.55219888e-03, -1.47054419e-01]],
 
        [[-6.38278574e-02, -2.71720346e-02,  3.16900611e-02, ...,
          -8.55979845e-02,  4.52550463e-02, -6.42086938e-02],
  

In [20]:
model.output[:,0]

<tf.Tensor 'strided_slice_4:0' shape=(None,) dtype=float32>

### make input 

In [2]:
val_base_dir =  "/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/engineered-data/totalSet/2020-11-09/small_splitted/test.tfrecord/part-r-"
val_data_dir = [val_base_dir+str(idx).zfill(5) for idx in range(5)]

In [99]:
import numpy as np

In [10]:
engineered_char = []
engineered_ngram=[]
engineered_word=[]
for x,_ in input_small_fn(val_data_dir, 200, 200, 20):
    engineered_char.append(x["char_input"].numpy())
    engineered_word.append(x["word_input"].numpy())
    engineered_ngram.append(x["ngram_input"].numpy())

In [100]:
url_list=[]
label=[]
source=[]
for x in get_label_url_source(val_data_dir):
    url_list.append(x["url"].numpy().decode("utf-8"))
    label.append(np.argmax(x["label"].numpy()))
    source.append(x["urlSource"].numpy().decode("utf-8"))

In [16]:
len(engineered_char)

10

In [10]:
import tensorflow as tf 

output_0 = model.output[:,0]
output_1 = model.output[:,1]

with tf.GradientTape() as tape:
    loss = model.output[:,0]
    conv_output = model.get_layer("word_layer_fsize_3").weights
grads = tape.gradient(loss, conv_output)[0]    

In [11]:
def read_tfrecord_v2(path, maxWordPerUrl, maxCharPerUrl, maxNgramPerUrl):
    dataset = tensorflow.data.TFRecordDataset(path)
    feature_map = {
        "engineeredChar": tensorflow.io.FixedLenFeature((maxCharPerUrl,), tensorflow.int64),
        "engineeredWord": tensorflow.io.FixedLenFeature((maxWordPerUrl,), tensorflow.int64),
        "engineeredNgram": tensorflow.io.FixedLenFeature((maxWordPerUrl * maxNgramPerUrl,), tensorflow.int64),
        "label": tensorflow.io.FixedLenFeature([], tensorflow.int64)
    }
    def _parse_fn(record, feature_map):
        example = tensorflow.io.parse_single_example(serialized=record, features=feature_map)
        onehot_label = tensorflow.cast(tensorflow.one_hot(tensorflow.where(tensorflow.equal(example["label"], 1), 1, 0), depth=2), dtype=tensorflow.int64)
        return {"char_input": example["engineeredChar"], "word_input": example["engineeredWord"],
                "ngram_input": example["engineeredNgram"]}, onehot_label
    dataset = dataset.batch(1).map(lambda record: _parse_fn(record, feature_map)).make_one_shot_iterator()
    return dataset

In [29]:
model.inputs

[<tf.Tensor 'char_input_1:0' shape=(None, 200) dtype=float32>,
 <tf.Tensor 'word_input_1:0' shape=(None, 200) dtype=float32>,
 <tf.Tensor 'ngram_input_1:0' shape=(None, 4000) dtype=float32>]

In [11]:
engineered_word[0:3]

[array([4170, 1199, 6133,   10, 2103, 8856,   10, 5292,   10,   10,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [34]:
import tensorflow as tf 
char_tensor = tf.convert_to_tensor(engineered_char)
ngram_tensor = tf.convert_to_tensor(engineered_ngram)
word_tensor = tf.convert_to_tensor(engineered_word)

In [72]:
dd = [char_tensor, ngram_tensor, word_tensor]
tf.concat(dd, axis=1)

<tf.Tensor: shape=(10, 4400), dtype=int32, numpy=
array([[56, 68, 68, ...,  0,  0,  0],
       [56, 68, 68, ...,  0,  0,  0],
       [56, 68, 68, ...,  0,  0,  0],
       ...,
       [56, 68, 68, ...,  0,  0,  0],
       [56, 68, 68, ...,  0,  0,  0],
       [56, 68, 68, ...,  0,  0,  0]], dtype=int32)>

In [28]:
tot_input = {"char_input":char_tensor, "word_input":word_tensor, "ngram_tensor":ngram_tensor}

In [39]:
tot_input = [char_tensor, ngram_tensor, word_tensor]
tot_input

[<tf.Tensor: shape=(10, 200), dtype=int32, numpy=
 array([[56, 68, 68, ...,  0,  0,  0],
        [56, 68, 68, ...,  0,  0,  0],
        [56, 68, 68, ...,  0,  0,  0],
        ...,
        [56, 68, 68, ...,  0,  0,  0],
        [56, 68, 68, ...,  0,  0,  0],
        [56, 68, 68, ...,  0,  0,  0]], dtype=int32)>,
 <tf.Tensor: shape=(10, 4000), dtype=int32, numpy=
 array([[56, 68, 68, ...,  0,  0,  0],
        [56, 68, 68, ...,  0,  0,  0],
        [56, 68, 68, ...,  0,  0,  0],
        ...,
        [56, 68, 68, ...,  0,  0,  0],
        [56, 68, 68, ...,  0,  0,  0],
        [56, 68, 68, ...,  0,  0,  0]], dtype=int32)>,
 <tf.Tensor: shape=(10, 200), dtype=int32, numpy=
 array([[ 4170,  1199,  6133, ...,     0,     0,     0],
        [ 4170, 10182,    10, ...,     0,     0,     0],
        [ 4170,  2547,    21, ...,     0,     0,     0],
        ...,
        [ 4170,    10,  2926, ...,     0,     0,     0],
        [ 4170,  3202,    10, ...,     0,     0,     0],
        [ 4170,  6891,   

In [14]:
grad_model = tf.keras.models.Model(
    [model.inputs], [model.get_layer("word_layer_fsize_3").output, model.output]
)

In [37]:
tot_input

[<tf.Tensor: shape=(1, 10, 200), dtype=int32, numpy=
 array([[[56, 68, 68, ...,  0,  0,  0],
         [56, 68, 68, ...,  0,  0,  0],
         [56, 68, 68, ...,  0,  0,  0],
         ...,
         [56, 68, 68, ...,  0,  0,  0],
         [56, 68, 68, ...,  0,  0,  0],
         [56, 68, 68, ...,  0,  0,  0]]], dtype=int32)>,
 <tf.Tensor: shape=(1, 10, 4000), dtype=int32, numpy=
 array([[[56, 68, 68, ...,  0,  0,  0],
         [56, 68, 68, ...,  0,  0,  0],
         [56, 68, 68, ...,  0,  0,  0],
         ...,
         [56, 68, 68, ...,  0,  0,  0],
         [56, 68, 68, ...,  0,  0,  0],
         [56, 68, 68, ...,  0,  0,  0]]], dtype=int32)>,
 <tf.Tensor: shape=(1, 10, 200), dtype=int32, numpy=
 array([[[ 4170,  1199,  6133, ...,     0,     0,     0],
         [ 4170, 10182,    10, ...,     0,     0,     0],
         [ 4170,  2547,    21, ...,     0,     0,     0],
         ...,
         [ 4170,    10,  2926, ...,     0,     0,     0],
         [ 4170,  3202,    10, ...,     0,     0,   

In [59]:
model.inputs

[<tf.Tensor 'char_input:0' shape=(None, 200) dtype=float32>,
 <tf.Tensor 'word_input:0' shape=(None, 200) dtype=float32>,
 <tf.Tensor 'ngram_input:0' shape=(None, 4000) dtype=float32>]

In [125]:
def get_result(idx, label):
    with tf.GradientTape() as tape:
    # conv_outputs 는 198*256짜리 : kernel_size3으로하면 그렇다. 그러한 filter의 갯수가 256개가 있으므로.. 196*256
    # predictions 는 이게 정상인지 악성인지 . 
        conv_outputs, predictions = grad_model([char_tensor, word_tensor, ngram_tensor])
        loss = predictions[:, label]
    grads = tape.gradient(loss,conv_outputs) 
    guided_grads = tf.cast(conv_outputs > 0, "float32") * tf.cast(grads > 0, "float32") * grads
    weight = tf.reduce_sum(guided_grads, axis=(1,2))
    
    ttmp = tf.reduce_sum(tf.multiply(weight[idx], conv_outputs[idx]),axis=1)
    output = ttmp.numpy()
    print(url_list[idx], "  :  ", label[idx], "\n")
    for i in range(20):
        idx2 = engineered_word[idx][i]
        word = word_dict.get(idx2)
        prob = output[i]
        print(word," ",prob,"\n")
    

In [105]:
final_tmp = tf.reduce_sum(tf.multiply(weight[0], conv_outputs[0]),axis=1)


In [106]:
output = final_tmp.numpy()

In [107]:
print(url_list[0],"  :  ", label[0])

http://0c606n6zcfrbk6db5t.mailone.loan/fvae444mfa903   :   1


In [108]:
for i in range(20):
    idx = engineered_word[0][i]
    word = word_dict.get(idx)
    prob = output[i]
    print(word," ",prob,"\n")

http   0.5558802 

c   0.40524948 

n   0.41920495 

<UNK>   0.19065289 

db   0.21377629 

t   0.43992442 

<UNK>   0.46625742 

loan   0.5371562 

<UNK>   0.32146972 

<UNK>   0.3018909 

<PAD>   0.3249948 

<PAD>   0.3249948 

<PAD>   0.3249948 

<PAD>   0.3249948 

<PAD>   0.3249948 

<PAD>   0.3249948 

<PAD>   0.3249948 

<PAD>   0.3249948 

<PAD>   0.3249948 

<PAD>   0.3249948 



In [87]:
engineered_word[0]

array([4170, 1199, 6133,   10, 2103, 8856,   10, 5292,   10,   10,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [64]:
tf.stack([final_tmp], axis=2)

ValueError: axis = 2 not in [-2, 2)

In [44]:
guided_grads[0]

<tf.Tensor: shape=(198, 256), dtype=float32, numpy=
array([[ 0.       , -0.       ,  0.       , ...,  0.       ,  0.       ,
         0.0281344],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ..., -0.       ,  0.       ,
         0.       ],
       ...,
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]], dtype=float32)>

In [40]:
conv_outputs[0]

<tf.Tensor: shape=(198, 256), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , ..., 0.02633833, 0.        ,
        0.00408098],
       [0.        , 0.        , 0.        , ..., 0.00844492, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02721744, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.01036725, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.01036725, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.01036725, 0.        ,
        0.        ]], dtype=float32)>

In [38]:
guided_grads[0]

<tf.Tensor: shape=(198, 256), dtype=float32, numpy=
array([[ 0.       , -0.       ,  0.       , ...,  0.       ,  0.       ,
         0.0281344],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ..., -0.       ,  0.       ,
         0.       ],
       ...,
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]], dtype=float32)>

In [16]:
predictions


<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[9.7283053e-01, 2.7169401e-02],
       [6.0301521e-03, 9.9396992e-01],
       [2.5263358e-02, 9.7473663e-01],
       [3.3427373e-02, 9.6657258e-01],
       [3.2015406e-02, 9.6798462e-01],
       [1.8334357e-02, 9.8166567e-01],
       [4.2183723e-02, 9.5781630e-01],
       [4.2502719e-04, 9.9957496e-01],
       [5.6901835e-02, 9.4309813e-01],
       [1.7459923e-02, 9.8254007e-01]], dtype=float32)>

In [42]:
grads2 = tf.reduce_sum(grads[0], axis=0)

In [43]:
grads2

<tf.Tensor: shape=(256,), dtype=float32, numpy=
array([ 9.19401878e-04, -5.43291681e-03,  3.58202774e-03, -4.33791545e-04,
       -2.99719609e-02, -1.51862344e-02, -3.08527425e-03,  1.16565768e-02,
       -9.22885723e-03, -1.04158688e-02,  1.61156245e-02, -4.48019058e-03,
        1.19484197e-02,  1.46627296e-02, -8.97300988e-03, -1.50216389e-02,
       -5.10398485e-03, -5.48715703e-03, -3.22397910e-02, -5.64692914e-03,
       -1.08256321e-02, -2.27625621e-03,  3.43708228e-03,  7.31497817e-03,
       -1.33044561e-02,  3.59028019e-03, -3.55631951e-03, -1.89220253e-02,
        9.96373594e-04, -2.90575624e-03,  1.30825490e-02,  3.95794585e-03,
        1.05610443e-03, -1.01920823e-02,  3.84788052e-03,  6.50050538e-03,
       -1.78239564e-03, -7.58798700e-03,  1.51814043e-03, -1.65889866e-03,
        9.06379148e-03,  2.68663210e-03,  1.12077799e-02,  7.61941075e-03,
       -2.11057924e-02, -1.29729630e-02, -4.84679490e-02,  1.10691478e-02,
        1.46593899e-03, -7.29838386e-03,  4.92498092

In [81]:
tf.multiply(tf.reduce_sum(conv_outputs, axis=(1,)),tf.reduce_sum(grads, axis=(1,)))

<tf.Tensor: shape=(10, 256), dtype=float32, numpy=
array([[ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00, ...,
        -7.16841081e-03, -2.90070079e-06,  1.14815804e-04],
       [ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00, ...,
        -1.66194257e-03, -2.63019979e-06,  0.00000000e+00],
       [ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00, ...,
        -6.68529561e-03, -0.00000000e+00,  1.02998386e-03],
       ...,
       [ 1.11339011e-06, -1.45750323e-06,  0.00000000e+00, ...,
        -1.13655493e-04, -0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00, ...,
        -1.51291005e-02, -7.52252117e-06,  0.00000000e+00],
       [ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00, ...,
        -4.72998712e-03, -8.46520607e-07,  0.00000000e+00]], dtype=float32)>

In [58]:
#guided_grads = tf.cast(conv_outputs > 0, "float32") * tf.cast(grads > 0, "float32") * grads
guided_grads = tf.keras.layers.ReLU(tf.reduce_sum(tf.multiply(conv_outputs,grads),axis=1))
# guirded_grads 는 198 * 256 이다. 

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [51]:
mean_output = tf.reduce_sum(conv_outputs, axis=1)
guided_grads_2 = tf.reduce_sum(guided_grads, axis=1)